<a href="https://colab.research.google.com/github/VighneshAlevoor/ML-Feature-Engineering/blob/master/titanic_Fisher_score.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

from sklearn.model_selection import train_test_split
from sklearn.feature_selection import chi2
from sklearn.feature_selection import SelectKBest, SelectPercentile

In [2]:
df=pd.read_csv('titanic.csv')
df.head()

,survived,pclass,sex,age,sibsp,parch,fare,embarked,class,who,adult_male,deck,embark_town,alive,alone,Unnamed: 15
0,0,3,male,22.0,1,0,7.2500,S,Third,man,True,NaN,Southampton,no,False,NaN
1,1,1,female,38.0,1,0,71.2833,C,First,woman,False,C,Cherbourg,yes,False,NaN
2,1,3,female,26.0,0,0,7.9250,S,Third,woman,False,NaN,Southampton,yes,True,NaN
3,1,1,female,35.0,1,0,53.1000,S,First,woman,False,C,Southampton,yes,False,NaN
4,0,3,male,35.0,0,0,8.0500,S,Third,man,True,NaN,Southampton,no,True,NaN


In [3]:
df.dtypes

survived         int64
pclass           int64
sex             object
age            float64
sibsp            int64
parch            int64
fare           float64
embarked        object
class           object
who             object
adult_male        bool
deck            object
embark_town     object
alive           object
alone             bool
Unnamed: 15    float64
dtype: object

In [4]:
df['sex']=df['sex'].map({'male':1,'female':0})
df['sex']

0      1
1      0
2      0
3      0
4      1
      ..
886    1
887    0
888    0
889    1
890    1
Name: sex, Length: 891, dtype: int64

In [5]:
df['embarked'].value_counts()

S    644
C    168
Q     77
Name: embarked, dtype: int64

In [6]:
df['embarked']=df['embarked'].map({'S':1,'C':2,'Q':3})
df['embarked']

0      1.0
1      2.0
2      1.0
3      1.0
4      1.0
      ... 
886    1.0
887    1.0
888    1.0
889    2.0
890    3.0
Name: embarked, Length: 891, dtype: float64

In [8]:
# separate train and test sets
X_train, X_test, y_train, y_test = train_test_split(
    df[['pclass', 'sex', 'embarked']],
    df['survived'],
    test_size=0.3,
    random_state=0)

X_train.shape, X_test.shape

((623, 3), (268, 3))

In [9]:
# calculate the chi2 p_value between each of the variables
# and the target
# it returns 2 arrays, one contains the F-Scores which are then 
# evaluated against the chi2 distribution to obtain the pvalue
# the pvalues are in the second array, see below

f_score = chi2(X_train.fillna(0), y_train)
f_score

(array([21.61080949, 63.55447864,  2.06662873]),
 array([3.33964360e-06, 1.55992554e-15, 1.50554002e-01]))

In [10]:
# let's add the variable names and order it for clearer visualisation

pvalues = pd.Series(f_score[1])
pvalues.index = X_train.columns
pvalues.sort_values(ascending=False)

embarked    1.505540e-01
pclass      3.339644e-06
sex         1.559926e-15
dtype: float64

Keep in mind, that contrarily to MI, where we were interested in the higher MI values, for Fisher score, the smaller the p_value, the more significant the feature is to predict the target, in this case Survival in the titanic.

Then, from the above data, Sex is the most important feature, then PClass then Embarked.

Note: One thing to keep in mind when using Fisher score or univariate selection methods, is that in very big datasets, most of the features will show a small p_value, and therefore look like they are highly predictive. This is in fact an effect of the sample size. So care should be taken when selecting features using these procedures. An ultra tiny p_value does not highlight an ultra-important feature, it rather indicates that the dataset contains too many samples.

Finally, in this demonstration, I used chi2 over 3 categorical variables only. If the dataset contained several categorical variables, we could then combine this procedure with SelectKBest or SelectPercentile, as I did in the previous lecture.
